# Function Calling and Planners

When we know exactly which semantic function we want to invoke in our app, we can just invoke that function in our code. As we grow our library of interesting and useful semantic functions, we might not want a specific button in our UI to invoke each function. Wouldn't it be cool if the AI could determine which semantic function best fits the user's question?.

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.7.1"
#!import ../config/SettingsHelper.cs
using Microsoft.SemanticKernel;

MySettings settings = Settings.LoadFromFile();
IKernelBuilder builder = Kernel.CreateBuilder();
if (settings.Type == "azure")
    builder.AddAzureOpenAIChatCompletion(
        settings.AzureOpenAI.ChatDeployment, "gpt-35-turbo", settings.AzureOpenAI.Endpoint, settings.AzureOpenAI.ApiKey);
else
    builder.AddOpenAIChatCompletion(
        settings.OpenAI.Model, settings.OpenAI.ApiKey, settings.OpenAI.OrgId);
Kernel kernel = builder.Build();

# Function Calling

Function calling is an amazing capability that allows the LLM to choose - and invoke - the most probable plugin function you've loaded into the kernel.

Before you run the following code, open the CelebratePlugin.cs file in the Plugins folder. Here, you'll see more than one KernelFunction are defined (e.g. GenerateOccasionCard and GenerateGiftIdeas). When creating the plan, we'll first use the user's question to determine the best function to call. We'll then execute the function chosen by the AI. 

In [ ]:
using Azure.AI.OpenAI;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

#!import Plugins/CelebratePlugin.cs

// load the kernel up with multiple functions to choose from
kernel.ImportPluginFromType<CelebratePlugin>();

// Enable auto invocation of kernel functions
OpenAIPromptExecutionSettings settings = 
    new() { ToolCallBehavior = ToolCallBehavior.EnableKernelFunctions };

IChatCompletionService chatCompletionService = 
    kernel.GetRequiredService<IChatCompletionService>();

ChatHistory chatHistory = [];
chatHistory.AddUserMessage("I have no idea what to get my dad for father's day! help?");

OpenAIChatMessageContent result = 
    (OpenAIChatMessageContent)await chatCompletionService
    .GetChatMessageContentAsync(chatHistory, settings, kernel);
ChatCompletionsFunctionToolCall toolCall = 
    result.ToolCalls.OfType<ChatCompletionsFunctionToolCall>().First();

// if a valid function was found, execute it
chatHistory.Add(result);
kernel.Plugins.TryGetFunctionAndArguments(
    toolCall,
    out KernelFunction? pluginFunction,
    out KernelArguments? arguments);
string? content = await pluginFunction.InvokeAsync<string>(kernel, arguments);

Console.WriteLine(content);

I know the response is long, but I want you to see the plan generated.

The response you are looking for is all the way at the bottom where it generated a list of ideas for father's day gifts even though we didn't direct the kernel to invoke the GenerateGiftIdeas function. There are two AI calls happening here:
1. We pass the prompt and a list of our available plugins/functions to an instance of our kernel. The kernel takes care of reading the descriptions of each of our funtions to the AI and determine which function best fits the user's provided input.
2. The second call passes our Father's Day prompt to the GenerateGiftIdeas function - and no other function. The AI correctly sees this as the most appropriate function.

# Planners

Content not yet complete...

# Exercise

